# Black jack 게임의 몬테카를로 학습 ! 

Blackjack rule : 

     ace 카드는 1 또는 11 값을 갖고, 2~10 카드는 2~10의 값을 갖고, J,Q,K 카드는 10의 값을 갖는다. 
     카드 딜러는 나(agent)에게 2장, 딜러 2장의 카드를 갖는데 하나는 펼치고 하나는 덮어둔다. 
     
     게임 룰은 카드의 숫자 합이 21에 가까운 사람이 이긴다. 
     
     카드를 2장씩 갖고 있는 상태에서 내가‘hit’라고 말하면 딜러에게 카드를 추가로 받을 수 있고‘stand’라고 외치면 더 받지 않겠다고 딜러에게 말하는 것. 
     근데 카드를 계속 받아서 숫자합이 21을 넘는것을‘bust’라고 하며 무조건 게임을 지게된다. 
     ace카드는 게임의 유불리에 따라 1 또는 11로 사용할 수 있으며 ace카드를 11로 사용하면 이를 usable ace라고 한다.
     딜러는 카드를 2장만 갖고있으므로 당연히 ace를 11로 사용한다.
     
     딜러는 17이 될떄까지 카드를 받아야하며 17이 넘어가면 카드를 받을 수 없다. 

 Black jack 환경에서 최적 policy를 구해보자. 아래 프로그램에 의해 Blackjack 환경을 만들고 초기 state를 보면 3개의 값으로 만들어진 tuple 값을 보여주고있다. 
 
 (19,9,True) 중에 19은 내가 받은 2장의 카드 숫자 합이고, 2는 딜러가 가진 2장의 카드 중 펼쳐진 한 장이 9인 것을 말하며, 마지막 False는 agent가 ace를 usable ace인지를 말해주는 것이다.
 
 즉, 내가 받은 2장의 카드중 ace를 11로써 사용한다는 말. 나머지 한장은 8 카드임을 말한다.

In [16]:
import gym 
env = gym.make('Blackjack-v1')
env.reset()

(19, 9, True)

아래 프로그램과 같이 Blackjack의 action space는 이산형이며 0(stand) 또는 1(hit)를 갖는다.


In [18]:
print(env.action_space)

Discrete(2)


다음 프로그램은 state가 (7,4,False)이므로 당연하게 hit(1)를 할 것이며 env.step(action)에 의해 (next_state, reward, done, _)를 출력하고 있다. 즉, 다음 state는 (16,4,False)가 나타났고 reward = 0.0, 그리고 done은 게임의 승리여부를 나타낸다. 

done=True이면 승부가 났다는 의미로 agent가 이기면 reward = 1.0, 지면 reward = -1, 그리고 비기면 reward = 0.0을 반환한다. 아래 프로그램에서는 done=False이므로 승부가 나지 않아 reward= 0이고 agent는 (16,4,False)이므로 승부는 알 수 없다.  

In [52]:
import gym
env=gym.make('Blackjack-v1')
state = env.reset()

print(state)
action=1
print(env.step(action))


(7, 4, False)
((16, 4, False), 0.0, False, {})


다음 프로그램은 임이의 action을 취해 승부가 날 때까지 transition(state, action, reward)를 episode로 지정된 변수에 저장하는 프로그램이다. 두 번 만에 게임이 딜러의 승리로 끝이 났으며 agent는 연속하여 hit해서 

 $$ (12,8, False),  ~action :~1,~reward: ~0  => ~(19,8,False) ~ action:~ 1, ~reward =~ -1~=> ~the ~dealer~ wins$$
 
 으로 state가 변했다는 것을 보여주고 있다. 

In [7]:
import gym
env=gym.make('Blackjack-v1')
state = env.reset()

episode=[]

num_timesteps=20

for t in range(num_timesteps):
    random_action = env.action_space.sample()
    
    new_state, reward, done, _ = env.step(random_action)
    
    episode.append((state, random_action, reward))
    
    if done:
        break
        
    state = new_state
    
print(episode)

[((12, 8, False), 1, 0.0), ((19, 8, False), 1, -1.0)]



다음 프로그램은 Blackjack 환경에서 agent(나)의 승률을 최대로 하는 최적 policy를 구하기 위한 프로그램이다. 

해설을 추가하였으나 이 프로그램은 Ravichandiran(2020)의 프로그램이다.

아래 프로그램에서 defaultdict는 변수를 dictionary 데이터로 정의하고자 할 때 사용하는 함수로, 특히 프로그램에서 최초로 key가 나타날 때 key에 대응하는 값을 0으로 놓기 때문에 초깃값을 설정하는 효과를 갖고 있다.

이어지는 프로그램에 있는 epsilon_greedy() 함수에 의해 Q=defaultdict(float)는 

 $$ Q(state, action) = [{'(state,action)':float}] $$
 의 형식으로 지정하고, total_return = defaultdict(float)는 
 
 $$ total_return((state,action)) = [{'state,action)':float}] $$
 으로, 그리고 N = defaultdict(int) 는
 $$N(state,action) = [{'(state,action)':int}] $$
 
 로 형식을 지정하게 된다.
 Q,total_return, N의 key인 (state,action)이 처음 나왔을 떄 0 값을 지정하므로 모든 state-action 쌍의 Q, total_return, N의 초기치를 0으로 부여하는 것과 동일한 역할을 하게 된다. 
 
 
 
 

In [3]:
import gym
import pandas as pd
import random
from collections import defaultdict

env=gym.make("Blackjack-v1")

Q = defaultdict(float)
total_return = defaultdict(float)
N = defaultdict(int)

print(Q)
print(total_return)
print(N)

defaultdict(<class 'float'>, {})
defaultdict(<class 'float'>, {})
defaultdict(<class 'int'>, {})


다음 프로그램은 epsilon-greedy policy와 episode를 생성하는 프로그램이다.
0~1 사이 값으 Uniform 분포로부터 추출하여 0.2보다 작으면 임이의 action을 선택하고 그렇지 않으면 Q 함수로부터 greedy하게 action을 한다.

$$max(list(range(env.action_ space.n)), key = lambda x:Q[(state,x)]) $$

는 key에 정의된 Q[(state,x)] 중 x를 기준으로 최대 Q를 정의하며 x는 list(range(env.action_space.n))에 읳 0,1이 차례로 입력된다. 

generate_episode( ) 함수에 의해 episode는 list 데이터로 정의되고 episode.append((state,action,reward))로 episode에 (state, action, reward)를 하나의 요소로 한 tuple 데이터가 저장된다. 



In [8]:
def epsilon_greedy(state,Q):
    epsilon = 0.2
    
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)), key = lambda x:Q[(state,x)])
    
num_timesteps=50
def generate_episode(Q):
    episode=[]
    state=env.reset()
    
    for t in range(num_timesteps):
        action= epsilon_greedy(state,Q)
        next_state, reward, done, _ = env.step(action)
        episode.append((state,action,reward))
        
        if done:
            break
        
        state = next_state
    return episode

    

끝으로 episode_greedy() 와 generate_episode() 함수를 이용해 500,000개의 episode를 생성해 최적 Q 함수를 구해보자. 
아래 프로그램에서 
$$ all~state~action~pairs=[(s_0,a_0),(s_1,a_1),...,(s_{T-1},a_{T-1})]$$
이고
$$ rewards=[r_0,r_1,...,r_{T-1}] $$

이 두개의 tuple list는 episode마다 새로 만들어지며 Q(s,a)는 FVMC이다. 


In [9]:
num_episodes=500000

for i in range(num_episodes):
    episode = generate_episode(Q)
    all_state_action_pairs = [(s,a) for (s,a,r) in episode]
    rewards = [r for (s,a,r) in episode]
    
    for t,(state,action,_) in enumerate(episode):
        if not (state,action) in all_state_action_pairs[:t]:
            G = sum(rewards[t:])
            total_return[(state,action)] += G
            N[(state,action)] += 1
            Q[(state,action)] = total_return[(state,action)]/N[(state,action)]
    

dictionary 데이터인 Q를 DataFrame으로 전환해 최초 10개의 Q 값을 출력해 보면 다음과 같다. 

Q의 key가 (state,action)이므로 첫 번째 자료의  state는 (8,4,False)이며 action은 0이며 이에 대응하는 최적 Q 값은 -0.304878임을 보여주고 있다.

state( 8,4,False)에서 action 1(hit)가 최적 policy임을 보여주고 있다.


In [48]:
df=pd.DataFrame(Q.items(),columns=['state_action_pair',"Q_value"])

print('{}이 나온 횟수 :: '.format(df['state_action_pair'][0]),N[((df['state_action_pair'][0]))])

df.head(10)


((8, 4, False), 0)이 나온 횟수 ::  164


,state_action_pair,Q_value
0,"((8, 4, False), 0)",-0.304878
1,"((8, 4, False), 1)",-0.040755
2,"((15, 10, False), 0)",-0.581842
3,"((15, 10, False), 1)",-0.571466
4,"((21, 1, True), 0)",0.661070
5,"((21, 1, True), 1)",-0.217391
6,"((7, 10, False), 0)",-0.561086
7,"((7, 10, False), 1)",-0.420281
8,"((12, 10, False), 0)",-0.597938
9,"((12, 10, False), 1)",-0.468646


아래와 같이 state가 (21,1,True) 일때 action = 0(stand)의 Q 값이 action=1(hit)일 때보다 크므로
action=0가 최적 policy, 즉 agent가 승리할 확률이 높고,


state가 (12,10,False) 일 때는 Q 값이 음의 값이므로 agent가 질 가능성이 높지만 
action=1(hit)이 최적임을 보여주고 있다. 
